In [ ]:
from datetime import date
from itertools import product

import cloudpickle
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

from app.model import DispatchConfig
from run_dispatch import run_dispatch

In [ ]:
DATES = pd.date_range(
    start="2024-01-01",
    end="2024-10-15",
    inclusive="both"
)
# DATES = [
#     pd.Timestamp("2024-03-08"),
# ]
# DATES = [
    # pd.Timestamp("2024-05-30"), #Cluster 1
    # pd.Timestamp("2024-10-07"), #Cluster 2
    # pd.Timestamp("2024-04-06"), #Cluster 3
    # pd.Timestamp("2024-04-26"), #Cluster 4
    # pd.Timestamp("2024-04-29"), #Cluster 5
    # pd.Timestamp("2024-05-26"), #Cluster 6
    # pd.Timestamp("2024-09-30"), #Cluster 7
    # pd.Timestamp("2024-02-03"), #Cluster 8
# ]
CONFIGS = [
    "preideal",
    # "bess_preideal",
    "bess_preideal_resource",
]

BESS_VALUES = {
    "BESS_1": {
        "MWh_nom": 2000,
        "hours_to_deplete": 2,
        "efficiency": 0.9,
        "min_soc": 0.05,
        "max_soc": 0.98,
        "initial_soc": 0.05,
        "charge_bid": 0,
        "discharge_bid": 0,
    },
}


DERS_YEARS = [None]


In [ ]:
output_unstacked = pd.DataFrame()
# models = []
# dispatches_type = []
dates_ = []
# DERS_data = []
# DERS_names = []
obj_bess, obj_no_bess = {}, {}
for (date_, dispatch_type, DERS) in product(DATES,CONFIGS, DERS_YEARS):
    try:
        BESS = BESS_VALUES if "bess" in dispatch_type.lower() else None
        mpo, model,  pmax_new_resources, expansion_sources = run_dispatch(DispatchConfig(dispatch_type=dispatch_type), DISPATCH_DATE=date_.date(), show_figs=False, BESS=BESS, DERS=DERS)
        print(f"Objective function for {dispatch_type} case: {model._model.objective.expr():,.2f}")
        # dispatches_type.append(dispatch_type)
        with open(f"data/models/{date_.date()}_{dispatch_type}_2000MWh_1000MW.pkl", "wb") as f:
            cloudpickle.dump(model, f)
        # models.append(model)
        dates_.append(date_)
        if "bess" in dispatch_type.lower():
            obj_bess[date_] = model._model.objective.expr()
        if "bess" not in dispatch_type.lower():
            obj_no_bess[date_] = model._model.objective.expr()
        # DERS_data.append(pmax_new_resources)
        # DERS_names.append(expansion_sources)
        # mpo["dispatch_type"] = dispatch_typex
        output_unstacked = pd.concat([output_unstacked, mpo])
    except Exception as e:
        print(f"Error in {date_} - {dispatch_type}: {e}")
        continue
output = output_unstacked.stack().reset_index()
output.columns = ["datetime", "source", "MPO"]
output.drop_duplicates(inplace=True)

In [ ]:
welfare = {}
for date_ in DATES:
    print(f"Obj with bess: {-obj_bess[date_]:,.2f}")
    print(f"Obj with No bess : {obj_no_bess[date_]:,.2f}")
    welfare[date_] = obj_no_bess[date_] - abs(obj_bess[date_])
    



In [ ]:
print(f"{sum(welfare.values())*1E-6:,.2f} [MCOP]")
print(f"{sum(welfare.values())*1E-6/4500:,.2f} [MUSD]")